In [10]:
%timeit




In [3]:

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error
from  sklearn.preprocessing import *
from sklearn.linear_model import *
from xgboost import XGBRegressor
from sklearn import model_selection
from sklearn.utils import class_weight
import numpy as np
import pandas as pd
from numpy import mean, absolute, loadtxt
import xgboost as xgb




In [4]:
train = pd.read_parquet('ai-fall20/datasets/predict-future-sales/train_mod.parquet')
test = pd.read_parquet('ai-fall20/datasets/predict-future-sales/test_mod.parquet')

In [5]:
train.head()

,shop_id,item_id,date_block_num,month,year,categories,GDP,GDP_change,CPI,avg_basket,monthly_sales
0,54,0,20,1,2014,40,64.1,0.7,11.4,552.7,1.0
1,55,1,15,4,2014,76,64.1,0.7,11.4,552.7,2.0
2,55,1,18,6,2014,76,64.1,0.7,11.4,552.7,1.0
3,55,1,19,4,2014,76,64.1,0.7,11.4,552.7,1.0
4,55,1,20,11,2014,76,64.1,0.7,11.4,552.7,1.0


In [6]:
test.head()

,shop_id,item_id,date_block_num,month,year,categories,GDP,GDP_change,CPI,avg_basket
0,5,5037,34,11,2015,19,62.4,-2.5,12.9,575.3
1,5,5320,34,11,2015,55,62.4,-2.5,12.9,575.3
2,5,5233,34,11,2015,19,62.4,-2.5,12.9,575.3
3,5,5232,34,11,2015,23,62.4,-2.5,12.9,575.3
4,5,5268,34,11,2015,20,62.4,-2.5,12.9,575.3


In [7]:
X = train.drop('monthly_sales', axis=1)
y = train.monthly_sales
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 2)



In [8]:
D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label= y_test)

D_max = xgb.DMatrix(X, label= y)


In [7]:
X_Train = train.drop('monthly_sales', axis=1)
Y = train.monthly_sales

In [7]:
model = XGBRegressor(tree_method='gpu_hist', gpu_id=0, booster='dart',objective='reg:squarederror', random_state=2)
model.fit(X_train, y_train)
y_pred = (model.predict(X_test))
print (y_pred)
mean_squared_error(y_test, y_pred, squared=False)


[1.890708  1.8247969 1.0597059 ... 1.9798101 2.1800852 2.091084 ]


5.606980262197854

In [8]:
xgb = XGBRegressor(tree_method='gpu_hist', gpu_id=0, booster='dart',objective='reg:squarederror', random_state=2, min_child_weight=1)

model = TransformedTargetRegressor(regressor=xgb, transformer=RobustScaler())
model.fit(X_train, y_train)
y_pred = (model.predict(X_test))


print (y_pred)
mean_squared_error(y_test, y_pred, squared=False)


[1.890718  1.8248005 1.0597084 ... 1.9798207 2.1800852 2.0910723]


5.596025603529435

In [12]:
param = {
    'tree_method':'gpu_hist',
    'gpu_id' : 0,
    'booster':'dart',
    'objective': 'reg:squarederror',
    'random_state': 2,

}

epochs = 20 
xmodel = xgb.train(param, D_train)
preds = xmodel.predict(D_test)
mean_squared_error(y_test, preds, squared=False)



6.1804058519639575

In [13]:
y_pred2 = np.rint(y_pred)
y_pred2

array([2., 2., 1., ..., 2., 2., 2.], dtype=float32)

In [ ]:
# buildsubmission
# for final submission

submission = pd.DataFrame()
submission['ID'] = test['ID'].tolist()
submission['item_cnt_month'] = y_pred

submission.head()




In [ ]:
submission.to_csv("ai-fall20/datasetssubmission.csv", index=False)

In [48]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
cv = KFold(n_splits=5, shuffle=True, random_state=2)

#the parameters can be built from the testing that goes on below
model = XGBRegressor(tree_method='gpu_hist', gpu_id=0,booster='dart',objective='reg:squarederror', random_state=2)
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=kfold, n_jobs=-1)

# convert scores to positive
scores = absolute(scores)
print(scores)

# summarize the result
s_mean = mean(scores)
print('Mean MAE: %.3f' % (s_mean))




D:\miniconda3\envs\strive\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [14]:
# without pipeline
min = MinMaxScaler()
lr = LinearRegression()

X_train = min.fit_transform(X_train)
X_test =  min.transform(X_test)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)




8.545719516626946

In [ ]:
catdf = pd.read_csv('ai-fall20/datasets/predict-future-sales/item_categories_translated.csv')
shopdf = pd.read_csv('ai-fall20/datasets/predict-future-sales/shops_translate.csv')
df =pd.read_csv('ai-fall20/datasets/predict-future-sales/Train_mod.csv', nrows=1500)

df.head(20)


In [15]:
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=1)
cv = KFold(n_splits=10, shuffle=True, random_state=1)

xgr = XGBRegressor(tree_method='gpu_hist', gpu_id=0,booster='dart', objective='reg:squarederror', random_state=2, max_depth=2, min_child_weight=1 )
scores = cross_val_score(xgr, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=1)
s_mean = mean(scores)
print('Mean MAE: %.3f' % (s_mean))



Mean MAE: -1.552


In [15]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV,RandomizedSearchCV


kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=2)

#the parameters can be built from the testing that goes on below
xgb = XGBRegressor(tree_method='gpu_hist', gpu_id=0,booster='dart',objective='reg:squarederror', random_state=2)


xgb.fit(X_train, y_train)
y_prediction = xgb.predict(X_test)


scores = cross_val_score(xgb, X,y, cv=kfold)
print('Accuracy:', np.round(scores, 2))
print('Acurracy mean: %0.2f' % (scores.mean()))

scores = cross_val_score(xgr, X, y, scoring='neg_mean_absolute_error', cv=kfold, n_jobs=-1)
s_mean = mean(scores)

print('Mean MAE: %.3f' % (s_mean))





D:\miniconda3\envs\strive\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"
Accuracy: [0.51 0.55 0.54]
Acurracy mean: 0.54
D:\miniconda3\envs\strive\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"
Mean MAE: -1.551


### This function along with Kfold will help fine tune parameters

In [9]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV,RandomizedSearchCV
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)



def grid_search(params, random=False): 
    xgr = XGBRegressor(tree_method='gpu_hist', gpu_id=0,booster='dart', objective='reg:squarederror', random_state=2,max_depth=6, learning_rate=0.6, gamma=1 )
    if random:
        grid = RandomizedSearchCV(xgr, params, cv=kfold, n_iter=20, n_jobs=1, verbose=10)
    else:
        grid = GridSearchCV(xgr, params, cv=kfold, n_jobs=1, verbose=10)

    grid.fit(X_train, y_train)
    best_params = grid.best_params_
    print("Best params:", best_params)
    best_score = grid.best_score_
    print("Training score: {:.3f}".format(best_score))


    scores = cross_val_score(xgr, X,y, cv=kfold)
    print('Accuracy:', np.round(scores, 2))
    print('Acurracy mean: %0.2f' % (scores.mean()))

    scores = cross_val_score(xgr, X, y, scoring='neg_mean_absolute_error', cv=kfold, n_jobs=1)
    s_mean = mean(scores)

    print('Mean MAE: %.3f' % (s_mean))

### start fine tuning

In [24]:
grid_search(params={'n_estimators':[100,200,400,800]},random=False
#800

Fitting 5 folds for each of 4 candidates, totalling 20 fits
D:\miniconda3\envs\strive\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
exception calling callback for <Future at 0x22312e81ee0 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "D:\miniconda3\envs\strive\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "D:\miniconda3\envs\strive\lib\site-packages\joblib\parallel.py", line 359, in __call__
    self.parallel.dispatch_next()
  File "D:\miniconda3\envs\strive\lib\site-packages\joblib\parallel.py", line 792, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "D:\miniconda3\envs\strive\lib\site-packages\joblib\parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "

KeyboardInterrupt: 

In [26]:
grid_search(params={'max_depth':[2, 3, 5, 6, 8]})

D:\miniconda3\envs\strive\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 5 is smaller than n_iter=20. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
D:\miniconda3\envs\strive\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END ....................................max_depth=2; total time=  11.2s
[CV 2/5] END ....................................max_depth=2; total time=  11.2s
[CV 3/5] END ....................................max_depth=2; total time=  12.0s
[CV 4/5] END ....................................max_depth=2; total time=  12.4s
[CV 5/5] END ....................................max_depth=2; total time=  12.9s
[CV 1/5] END ....................................max_dept

KeyboardInterrupt: 

In [2]:
%timeit
grid_search(params={'booster':['dart', 'gbtree', 'gblinear']})

NameError: name 'grid_search' is not defined

In [12]:
%timeit
grid_search(params={'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5], 'max_depth':[6]}, random=True)

D:\miniconda3\envs\strive\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 7 is smaller than n_iter=20. Running 7 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
D:\miniconda3\envs\strive\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
Fitting 5 folds for each of 7 candidates, totalling 35 fits
[CV 1/5; 1/7] START learning_rate=0.01, max_depth=6.............................
[CV 1/5; 1/7] END ...........learning_rate=0.01, max_depth=6; total time=  23.0s
[CV 2/5; 1/7] START learning_rate=0.01, max_depth=6.............................
[CV 2/5; 1/7] END ...........learning_rate=0.01, max_depth=6; total time=  22.6s
[CV 3/5; 1/7] START learning_rate=0.01, max_depth=6.............................
[CV 3/5; 1/7] END ...........learning_rate=0.01, max_dept

In [13]:
grid_search(params={'gamma':[0, 0.1, 0.5, 1, 2, 5]})

Fitting 5 folds for each of 6 candidates, totalling 30 fits
D:\miniconda3\envs\strive\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
[CV 1/5; 1/6] START gamma=0.....................................................
[CV 1/5; 1/6] END ...................................gamma=0; total time=  28.8s
[CV 2/5; 1/6] START gamma=0.....................................................
[CV 2/5; 1/6] END ...................................gamma=0; total time=  28.7s
[CV 3/5; 1/6] START gamma=0.....................................................
[CV 3/5; 1/6] END ...................................gamma=0; total time=  30.4s
[CV 4/5; 1/6] START gamma=0.....................................................
[CV 4/5; 1/6] END ...................................gamma=0; total time=  29.6s
[CV 5/5; 1/6] START gamma=0.........................

In [14]:
grid_search(params={'min_child_weight':[1, 2, 3, 4, 5]})

Fitting 5 folds for each of 5 candidates, totalling 25 fits
D:\miniconda3\envs\strive\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
[CV 1/5; 1/5] START min_child_weight=1..........................................
[CV 1/5; 1/5] END ........................min_child_weight=1; total time=  29.7s
[CV 2/5; 1/5] START min_child_weight=1..........................................
[CV 2/5; 1/5] END ........................min_child_weight=1; total time=  30.1s
[CV 3/5; 1/5] START min_child_weight=1..........................................
[CV 3/5; 1/5] END ........................min_child_weight=1; total time=  29.9s
[CV 4/5; 1/5] START min_child_weight=1..........................................
[CV 4/5; 1/5] END ........................min_child_weight=1; total time=  28.8s
[CV 5/5; 1/5] START min_child_weight=1..............

In [15]:
grid_search(params={'subsample':[0.5, 0.7, 0.8, 0.9, 1]})

Fitting 5 folds for each of 5 candidates, totalling 25 fits
D:\miniconda3\envs\strive\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
[CV 1/5; 1/5] START subsample=0.5...............................................
[CV 1/5; 1/5] END .............................subsample=0.5; total time=  29.4s
[CV 2/5; 1/5] START subsample=0.5...............................................
[CV 2/5; 1/5] END .............................subsample=0.5; total time=  31.7s
[CV 3/5; 1/5] START subsample=0.5...............................................
[CV 3/5; 1/5] END .............................subsample=0.5; total time=  30.4s
[CV 4/5; 1/5] START subsample=0.5...............................................
[CV 4/5; 1/5] END .............................subsample=0.5; total time=  31.2s
[CV 5/5; 1/5] START subsample=0.5...................

In [ ]:
grid_search(params={'colsample_bytree':[0.5, 0.7, 0.8, 0.9, 1]})

In [ ]:
grid_search(params={'n_estimators':[2, 25, 50, 75, 100]})

In [ ]:
grid_search(params={'max_depth':[1, 2, 3, 4, 6, 7, 8], 'n_estimators':[2, 50, 100]})

In [ ]:
grid_search(params={'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5], 'max_depth':[6], 'n_estimators':[50]})

In [ ]:
grid_search(params={'min_child_weight':[1, 2, 3, 4, 5], 'max_depth':[2], 'n_estimators':[50]})

In [ ]:
grid_search(params={'subsample':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 'max_depth':[2], 'n_estimators':[50]})

In [ ]:
grid_search(params={'subsample':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 
                    'min_child_weight':[1, 2, 3, 4, 5], 
                    'learning_rate':[0.1, 0.2, 0.3, 0.4, 0.5], 
                    'max_depth':[1, 2, 3, 4, 5, None], 
                    'n_estimators':[2, 25, 50, 75, 100]},
                    random=True)

In [ ]:
grid_search(params={'colsample_bynode':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 'colsample_bylevel':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 'colsample_bytree':[0.5, 0.6, 0.7, 0.8, 0.9, 1], 'max_depth':[2], 'n_estimators':[25,50]})

In [ ]:
grid_search(params={'gamma':[0, 0.01, 0.05, 0.1, 0.5, 1, 2, 3], 'colsample_bylevel':[0.7], 'colsample_bytree':[0.8], 'colsample_bynode':[1], 'max_depth':[2], 'n_estimators':[50]})

In [ ]:
model = XGBRegressor(booster='dart', objective='reg:squarederror', random_state=2, n_estimators=5000)
eval_set = [(X_test, y_test)]
eval_metric='error'
model.fit(X_train, y_train, eval_metric="error", eval_set=eval_set, early_stopping_rounds=500)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(model.get_booster().best_iteration)

## The two cells are necessary 
### The first cell prepares the data and after you input the data in your model the y_prediction varaible is saved
### The second cell creates the CSV to be submitted 

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

#the parameters can be built from the testing that goes on below
xgb = XGBClassifier(booster='gbtree', objective='binary:logistic', max_depth=2, min_child_weight=3,subsample=0.6, learning_rate=0.2, n_estimators=50, random_state=2, n_jobs=-1,colsample_bylevel = 0.7, colsample_bynode =1, colsample_bytree = 0.8, verbosity=0, gamma=0)


xgb.fit(X_train, y_train)
y_prediction = xgb.predict(X_test)


scores = cross_val_score(xgb, X,y, cv=kfold)
print(y_prediction)

print('Accuracy:', np.round(scores, 2))
print('Acurracy mean: %0.2f' % (scores.mean()))




### Voting 

In [ ]:
from sklearn.ensemble import VotingClassifier


estimator = []
estimator.append(('lr', LogisticRegression(solver = 'lbfgs', multi_class = 'multinomial', max_iter = 200, random_state=1)))
estimator.append (('rf',RandomForestClassifier(random_state=1)))
estimator.append(('svc',SVC(kernel='rbf', gamma = 'auto', probability= True)))
estimator.append(('kn', KNeighborsClassifier()))
estimator.append(('gbc',GradientBoostingClassifier()))
estimator.append(('gnb',GaussianNB()))
estimator.append(('xg', XGBClassifier(booster='gbtree', objective='binary:logistic', max_depth=2, min_child_weight=3,subsample=0.6, learning_rate=0.2, n_estimators=50, random_state=2, n_jobs=-1,colsample_bylevel = 0.7, colsample_bynode =1, colsample_bytree = 0.8, verbosity=0, gamma=0)))

vc_hard = VotingClassifier(estimators=estimator, voting='hard',n_jobs=-1, verbose=True, weights=[1,2,2,2,2,1,3] )
vc_hard.fit(X_train,y_train)
vc_hard.score(X_test,y_test)
print('HARD SCORE')
y_prediction = vc_hard.predict(X_test)
moreinfo(vc_hard)

vc_soft = VotingClassifier(estimators=estimator, voting='soft', n_jobs=-1, verbose=True, weights=[1, 3, 2, 1, 2, 1, 2])
vc_soft.fit(X_train,y_train)
vc_soft.score(X_test,y_test)
print('SOFT SCORE')
y_prediction = vc_soft.predict(X_test)
moreinfo(vc_soft)








In [ ]:
from sklearn.ensemble import VotingClassifier


estimator = []
estimator.append(('lr', LogisticRegression(solver = 'lbfgs', multi_class = 'multinomial', max_iter = 200, random_state=1)))
estimator.append (('rf',RandomForestClassifier(random_state=1)))
estimator.append(('svc',SVC(kernel='rbf', gamma = 'auto', probability= True)))
estimator.append(('kn', KNeighborsClassifier()))
estimator.append(('gbc',GradientBoostingClassifier()))
estimator.append(('gnb',GaussianNB()))
estimator.append(('xg', XGBClassifier(booster='gbtree', objective='binary:logistic', max_depth=2, min_child_weight=3,subsample=0.6, learning_rate=0.2, n_estimators=50, random_state=2, n_jobs=-1,colsample_bylevel = 0.7, colsample_bynode =1, colsample_bytree = 0.8, verbosity=0, gamma=0)))


vc_soft = VotingClassifier(estimators=estimator, voting='soft', n_jobs=-1, verbose=True, weights=[1, 3, 2, 1, 2, 1, 2])
vc_soft.fit(X_train,y_train)
vc_soft.score(X_test,y_test)
print('SOFT SCORE')
y_prediction = vc_soft.predict(X_test)
moreinfo(vc_soft)


In [ ]:
params = {'weights': b_list}
kfold = StratifiedKFold(n_splits=8, shuffle=True, random_state=2)
rando = RandomizedSearchCV(vc_soft, params, cv=kfold, n_iter=20, n_jobs=-1)
rando.fit(X_train, y_train)
best_params = rando.best_params_
print("Best params:", best_params)
best_score = rando.best_score_
print(best_score)


In [ ]:
params = {'weights': b_list}
kfold = StratifiedKFold(n_splits=8, shuffle=True, random_state=2)
gs = GridSearchCV(vc_soft, param_grid=params, cv=kfold, n_jobs=-1)
gs.fit(X_train, y_train)
best_params = gs.best_params_
print("Best params:", best_params)
best_score = gs.best_score_
print(best_score)


In [ ]:
# build a list 


b_list = list()
for a in range(1,3):
    for b in range(1,5):
        for c in range(1,4):
            for d in range(1,5):
                for e in range(1,4):
                    for f in range(1,3):
                        for g in range(1,6):
                            a_list = [a,b,c,d,e,f,g]
                            b_list.append(a_list)

print(b_list)





### Stacking

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot
from sklearn.datasets import load_wine,load_iris
from matplotlib.pyplot import figure
figure(num=2, figsize=(16, 12), dpi=80, facecolor='w', edgecolor='k')
 
 
 
# get a stacking ensemble of models
def get_stacking():

  # define the base models
  level0 = list()
  level0.append(('lr', LogisticRegression()))
  level0.append(('knn', KNeighborsClassifier()))
#   level0.append(('cart', DecisionTreeClassifier()))
  level0.append(('svm', SVC(kernel='rbf')))
#   level0.append(('bayes', GaussianNB()))
  level0.append(('xgb', XGBClassifier(booster='gbtree', objective='binary:logistic', max_depth=2, min_child_weight=3,subsample=0.6, learning_rate=0.2, n_estimators=50, random_state=2, n_jobs=-1,colsample_bylevel = 0.7, colsample_bynode =1, colsample_bytree = 0.8, verbosity=0, gamma=0)))

  # define meta learner model
  level1 = SVC()
  
  # define the stacking ensemble
  model = StackingClassifier(estimators=level0, final_estimator=level1, cv=kfold)
  return model
 
# get a list of models to evaluate
def get_models():
  models = dict()
  models['LogisticRegression'] = LogisticRegression()
  models['KNeighborsClassifier'] = KNeighborsClassifier()
#   models['Decision tree'] = DecisionTreeClassifier()
  models['svm'] = SVC(kernel='rbf')
#   models['GaussianNB'] = GaussianNB()
  models['XGBoost'] = XGBClassifier(booster='gbtree', objective='binary:logistic', max_depth=2, min_child_weight=3,subsample=0.6, learning_rate=0.2, n_estimators=50, random_state=2, n_jobs=-1,colsample_bylevel = 0.7, colsample_bynode =1, colsample_bytree = 0.8, verbosity=0, gamma=0)
  models['stacking'] = get_stacking()
  return models
 
# evaluate a give model using cross-validation
def evaluate_model(model):
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
  return scores
 
# get the models to evaluate
models = get_models()


# evaluate the models and store results
results, names, results1 = list(), list(),list()
for name, model in models.items():
  scores= evaluate_model(model)
  results.append(scores)
 
  names.append(name)
  print('>%s -> %.3f (%.3f)---Titanic dataset' % (name, mean(scores), std(scores)))
  
# plot model performance for comparison
pyplot.rcParams["figure.figsize"] = (15,6)
pyplot.boxplot(results, labels=[s+"-Titanic" for s in names], showmeans=True)
pyplot.show()


In [ ]:

 
#base 
level0 = list()
level0.append(('lr', LogisticRegression()))
level0.append(('knn', KNeighborsClassifier()))
#level0.append(('cart', DecisionTreeClassifier()))
level0.append(('svm', SVC(kernel='rbf')))
# level0.append(('bayes', GaussianNB()))
level0.append(('xgb', XGBClassifier(booster='gbtree', objective='binary:logistic', max_depth=2, min_child_weight=3,subsample=0.6, learning_rate=0.2, n_estimators=50, random_state=2, n_jobs=-1,colsample_bylevel = 0.7, colsample_bynode =1, colsample_bytree = 0.8, verbosity=0, gamma=0)))

#second level
level1 = SVC()


cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
 
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=kfold) 
model.fit(X_train, y_train)
y_prediction = model.predict(X_test)

print(y_prediction)

print('Accuracy:', np.round(scores, 2))
print('Acurracy mean: %0.2f' % (scores.mean()))




In [51]:

from datetime import *
from dateutil.parser import parse
import numpy as np
import pandas as pd

#read data
data = pd.read_csv('/content/drive/MyDrive/ai-fall20/datasets/predict-future-sales/sales_train.csv',nrows = 1500)
tree = pd.read_csv('/content/drive/MyDrive/ai-fall20/datasets/predict-future-sales/items.csv', nrows=1500)
test = pd.read_csv('/content/drive/MyDrive/ai-fall20/datasets/predict-future-sales/test.csv', nrows=1500)


# Manipulate Train Data
from dateutil.parser import parse
data.date = data.date.astype('str')
data.date = data.date.apply(lambda x: parse(x))
data.date = data.date.apply(lambda x: pd.to_datetime(x,
    format='%Y-%m-%dT',
    errors='coerce'))
data['month'] = data.date.dt.month
data['year'] = data.date.dt.year
data = data.groupby(['item_id','shop_id','date_block_num'], as_index=False).agg({'item_price':'sum', "item_cnt_day":'sum','month':'first', 'year':'first'})
tree = tree.drop('item_name', axis=1)

# summed counts by month, change name 
data = data.rename(columns={'item_cnt_day':'monthly_sales'})
data = data.reset_index()
data = data.drop('index', axis=1)


# extra features
fin_map = {2013:[63.6,1.8,6.5,519.6], 2014:[64.1, 0.7, 11.4, 552.7], 2015:[62.4,-2.5, 12.9, 575.3]}

# Gist - SAVE!!! for now the only way I know how to create a dictionary with two columns
# to map to a dataframe
branch = tree.set_index('item_id').T.to_dict('records')[0]
data['categories'] = data['item_id'].map(branch)


# Assigning features from finance data
data['finance'] = data['year'].map(fin_map)
data[['GDP','GDP_change','CPI','avg_basket']] = pd.DataFrame(data.finance.tolist(), index= data.index)
test = test.assign(date_block_num = 34 , month = 11, year = 2015)
test['categories'] = test['item_id'].map(branch)
test = test.assign(GDP = fin_map[2015][0], GDP_change = fin_map[2015][1], CPI = fin_map[2015][2], avg_basket = fin_map[2015][3])
test = test.drop('ID', axis=1)
data = data.drop('finance', axis=1)

#export first csv for visualization
data.to_parquet("/content/drive/MyDrive/ai-fall20/datasets/predict-future-sales/dfviz.parquet", engine="pyarrow")


#last things to share csv, dropping and rearranging
col_rearrange = ['shop_id','item_id','date_block_num','month','year', 'categories', 'GDP','GDP_change','CPI','avg_basket','monthly_sales']
data = data[col_rearrange]
del col_rearrange[-1]
test = test[col_rearrange]


#WriteOut 
data.to_csv("/content/drive/MyDrive/ai-fall20/datasets/predict-future-sales/Train_mod.csv", index=False)
test.to_csv("/content/drive/MyDrive/ai-fall20/datasets/predict-future-sales/Test_mod.csv", index=False)
data.to_parquet("/content/drive/MyDrive/ai-fall20/datasets/predict-future-sales/train_mod.parquet", engine="pyarrow")
test.to_parquet("/content/drive/MyDrive/ai-fall20/datasets/predict-future-sales/test_mod.parquet", engine="pyarrow")

time: 174 ms


In [52]:
test.head()

,shop_id,item_id,date_block_num,month,year,categories,GDP,GDP_change,CPI,avg_basket
0,5,5037,34,11,2015,NaN,62.4,-2.5,12.9,575.3
1,5,5320,34,11,2015,NaN,62.4,-2.5,12.9,575.3
2,5,5233,34,11,2015,NaN,62.4,-2.5,12.9,575.3
3,5,5232,34,11,2015,NaN,62.4,-2.5,12.9,575.3
4,5,5268,34,11,2015,NaN,62.4,-2.5,12.9,575.3


time: 10.5 ms


In [53]:
data.head()

,shop_id,item_id,date_block_num,month,year,categories,GDP,GDP_change,CPI,avg_basket,monthly_sales
0,25,785,0,1,2013,49.0,63.6,1.8,6.5,519.6,3.0
1,25,791,0,2,2013,73.0,63.6,1.8,6.5,519.6,8.0
2,25,804,0,1,2013,49.0,63.6,1.8,6.5,519.6,1.0
3,25,810,0,1,2013,77.0,63.6,1.8,6.5,519.6,1.0
4,25,829,0,1,2013,73.0,63.6,1.8,6.5,519.6,1.0


time: 10.7 ms
